### GAIA optimiation DBSCAN with MCMC scheme

1. using a simple ABC scheme
2. ...

In [1]:

using PyCall
using Distributions
using Statistics , StatsBase

using Distances
using Random

push!(LOAD_PATH,"/home/stephane/Science/cluster/GAIA/master/src")
using GaiaClustering

import PyPlot


## directory
rootdir = "/home/stephane/Science/cluster/GAIA"
wdir    = "/home/stephane/Science/cluster/GAIA/products"

cd(wdir)


#### mutable MC and model

mutable struct mc
    eps::Array{Float64}
    mne::Array{Int32}
    mcl::Array{Int32}
    qc::Array{Float64}
    qn::Array{Int32}
end

mutable struct model
    eps::Float64
    min_nei::Int
    min_cl::Int
end

struct abc
    minQ::Float64
    minstars::Int
    epsmean::Float64
    epsdisp::Float64
    min_nei::Int
    min_cl::Int
    ncoredisp::Int
    nburnout::Int
    niter::Int
end

In [2]:
### generate eps, min_neightbor and min_cluster
###
### aeps,amin_nghb, amin_cluster are the parameter of the prior density for the DBSCAN parameters

function theta(p::abc)
    peps         = TruncatedNormal(p.epsmean, epsdisp, 0.1 , 1000)
    pminnei      = TruncatedNormal(p.min_nei, p.ncoredisp, 1 , 1000.)
    pmincl      = TruncatedNormal(p.min_cl, p.ncoredisp, 1 , 1000.)
    
    e = rand(peps)          ; pe = pdf(peps,e)
    n = trunc(Int,rand(pminnei))  ; pn = pdf(pminnei, n)
    c = trunc(Int,rand(pmincl))   ; pc = pdf(pmincl, c)
    
    ptotal = pe*pn*pc
    params = model(e, n, c)
    return(params, ptotal)
end


## iterate with random walk and yield the probability
##
function thetaiter(θi::model , p::abc)
    let   
        peps         = TruncatedNormal(p.epsmean, epsdisp, 0.1 , 1000)
        pminnei      = TruncatedNormal(p.min_nei, p.ncoredisp, 1 , 1000.)
        pmincl       = TruncatedNormal(p.min_cl, p.ncoredisp, 1 , 1000.)
        
        eps_rw    = TruncatedNormal(θi.eps,0.5 , 0.1, 1000.)
        minnei_rw = TruncatedNormal(θi.min_nei , 4  , 1,  1000.)
        mincl_rw  = TruncatedNormal(θi.min_cl  , 4  , 1, 1000.)
        
        new_e = 0.
        new_mn = 0
        new_mcl = 0
        
        iternotfound =true
        while iternotfound
            new_e = rand(eps_rw)
            new_mn =  trunc(Int,rand(minnei_rw))      
            new_mcl = trunc(Int,rand(mincl_rw))      
        
            if new_e > 0 && new_mn > 0 && new_mcl > 0
                iternotfound = false
            end
        end
    
        pe = pdf(peps , new_e)
        pn = pdf(pminnei, new_mn)
        pc = pdf(pmincl, new_mcl)
        ptotal = pe * pn * pc
    
        params = model(new_e, new_mn, new_mcl)
        return(params, ptotal)
    end
end   

thetaiter (generic function with 1 method)

In [3]:
## compute the dbscan clusters, the metric and the Metropolis acceptance

function find_clusters(df::GaiaClustering.Df, dfcart::GaiaClustering.Df , m::model)
    let 
        labels = clusters(df.data , m.eps , 20, m.min_nei, m.min_cl)
        if length(labels) == 0
            return(0, 0)
        end
        
    ### metrics of the clusters
        q2d = metric(dfcart, labels, "spatial2d" , 2.0 , 20.0, 20 )
        qv = metric(dfcart, labels, "velocity" , 3.0 , 30.0, 20 )
        qp, qa = metric(dfcart, labels, "HRD" )
    
        nlab = []
        for ilab in labels
            push!(nlab,length(ilab))
        end
    
    
    #### metric for number of stars in the cluster
        qn = []
        for nl in nlab
            push!(qn,log10(nl))
        end
    
        qc = 0.
        qstar = 0
        for i in 1:length(nlab)
            k1 = q2d[i][1]
            k2 = qv[i][1]
            k3 = qa[i][1]
            k4 = qn[i]
            qq = (2k1 + 3k2 + k3 + k4) / 7.0
            if qq > qc 
                qc = qq
                qstar = nlab[i]
            end
        end
        return(qc, qstar)
    end
end

find_clusters (generic function with 1 method)

In [4]:
function getdata(filevot)
    voname = filevot

    data       = read_votable(voname)
    df         = filter_data(data)
    dfcart     = add_cartesian(df)
    blck       = [[1,2,3],[4,5], [6,7,8]]
    wghtblck   = [4.0,5.0,1.0]
    norm       = "identity"

    dfcartnorm , scale8 = normalization_PerBlock(dfcart, blck, wghtblck , norm, false) 
    return(dfcartnorm , dfcart)
end

getdata (generic function with 1 method)

In [5]:
## mini stats over Qc and Qn
function ministats(niter::Int, df::GaiaClustering.Df, dfcart::GaiaClustering.Df,mi::model, params::abc)
    println("### mini stats...")
    qcmini = []
    qnmini = []
    for i in 1:niter
        mitest, probtest = thetaiter(mi , params)
        qtest , ntest = find_clusters(df, dfcart, mitest)
        push!(qcmini,qtest)
        push!(qnmini,ntest)
    end
    println("### Qc : ",mean(qcmini))
    println("### Qn : ",mean(qnmini))    
    
    return(0)
end

ministats (generic function with 1 method)

In [6]:
## ABC MCMC (following Weyan et al. 2013)
## first simple scheme for testing
##
function abc_mcmc(df::GaiaClustering.Df, dfcart::GaiaClustering.Df, params::abc)
    let 
        mci = mc(zeros(Float64,0),zeros(Int32,0),zeros(Int32,0) , zeros(Float64,0), zeros(Int32,0))
        
        initial = true
        th = []
        mi = model(0.0,0,0)
        micurrent = model(0.0,0,0)
        probi = 0.
        probcurrent = 0.
    
        minimumQ = params.minQ
        minstars = params.minstars
        niter = params.niter
        nburn = params.nburnout
    
        
        while initial
            mi, probi = theta(params)
            qres , nstars = find_clusters(df, dfcart, mi)
            if qres > minimumQ && nstars >= minstars 
                println("### init done ...")
                initial = false
                push!(mci.eps, mi.eps)
                push!(mci.mne, mi.min_nei)
                push!(mci.mcl, mi.min_cl)
                push!(mci.qc,  qres)
                push!(mci.qn,  nstars)
            end
        end

        ministats(100, df, dfcart, mi , params)
        
        nchain = 1
        loopAgain = true
        burndone = false
                    
        while loopAgain
            micurrent, probcurrent = thetaiter(mi , params)
            qres , nstars = find_clusters(df, dfcart, micurrent)
        
            if qres > minimumQ && nstars >= minstars 
            ### Metropolis-Hasting
                α = probcurrent / probi
                if α > rand() 
                    nchain += 1
                    if (nchain%500 == 0) println("### chain:",nchain) end
                    if nchain > nburn && !burndone println("### burnout done...") ; nchain = 0 ; burndone = true end
                    if nchain > niter loopAgain = false end
                    if burndone
                        mi = micurrent
                        push!(mci.eps, mi.eps)
                        push!(mci.mne, mi.min_nei)
                        push!(mci.mcl, mi.min_cl)
                        push!(mci.qc , qres)
                        push!(mci.qn,  nstars)
                    end
                end
            end
        end
        return(mci)
    end
end

abc_mcmc (generic function with 1 method)

## Main ...
## Votable test.vot read
## Filtering done ...
## Stars selected: 78163
## Cartesian transformation done ...
## Normalization identity done...
### [1pc,1pc,1pc,1km/s,1km/s,1mag,1mag] equivalent to [0.398015, 0.398015, 0.398015, 0.497519, 0.497519, 0.0995037, 0.0995037, 0.0995037]
## MCMC : Q minimum  = 2.4
## MCMC : N stars minimum = 40
## MCMC : iterations = 100
### init done ...
### mini stats...
### Qc : 3.9056460536572155
### Qn : 238.95
### burnout done...
## MCMC done.
## Stats:

ϵ : 2.371737782102198 +/- 0.8008860396675839
min_nei  : 6.825242718446602 +/- 3.876594382849157
min_clus  : 10.854368932038835+/- 3.175913538453338
Qn  : 293.80582524271847 +/- 313.822075800686
Qc  : 3.3931429456462867 +/- 0.2884572227929368


In [7]:
### main loop

function main(params::abc)
    Random.seed!(1237)
    println("## Main ...")
    
    ## Get the cluster data..
    dfnorm , dfcart = getdata("test.vot")
    
    ## run the mcmc for the dbscan parameters

    println("## MCMC : Q minimum  = ", params.minQ)
    println("## MCMC : N stars minimum = ", params.minstars)
    println("## MCMC : iterations = ", params.niter)
    mc = abc_mcmc(dfnorm, dfcart, params)
    
    println("## MCMC done.")
    
    ## plots ################################
    
    PyPlot.plt[:figure](figsize=(10.0,12.0))

    
    nbins = 50
    PyPlot.plt[:subplot](3, 2, 1  )
    h = PyPlot.plt[:hist](mc.eps,nbins, alpha=0.75)
    PyPlot.plt[:xlabel]("ϵ")
    PyPlot.plt[:grid](true)
    
    nbins = 20
    PyPlot.plt[:subplot](3, 2, 2 )
    h = PyPlot.plt[:hist](mc.mne,nbins, alpha=0.75)
    PyPlot.plt[:xlabel]("min_neighbor")  
    PyPlot.plt[:grid](true)
    
    nbins = 20
    PyPlot.plt[:subplot](3, 2, 3 )
    h = PyPlot.plt[:hist](mc.mcl,nbins, alpha=0.75)
    PyPlot.plt[:xlabel]("min_cluster") 
    PyPlot.plt[:grid](true)
    
    ### text 
    PyPlot.plt[:subplot](3, 2, 4)
    PyPlot.plt[:axis]("off")
    vtext  = mean(mc.eps)
    v2text = std(mc.eps)
    txt = "ϵ : $vtext +/- $v2text "
    PyPlot.plt[:text](0.02,0.1,txt)
    
    vtext  = mean(mc.mne)
    v2text = std(mc.mne)
    txt = "min_neigh : $vtext +/- $v2text "
    PyPlot.plt[:text](0.02,0.25,txt)  
    
    vtext  = mean(mc.mcl)
    v2text = std(mc.mcl)
    txt = "min_clus : $vtext +/- $v2text "
    PyPlot.plt[:text](0.02,0.4,txt)    
    
    vtext  = mean(mc.qn)
    v2text = std(mc.qn)
    txt = "Qn : $vtext +/- $v2text "
    PyPlot.plt[:text](0.02,0.55,txt)
    
    vtext  = mean(mc.qc)
    v2text = std(mc.qc)
    txt = "Qc : $vtext +/- $v2text "
    PyPlot.plt[:text](0.02,0.7,txt)
    
    #######
    
    nbins = 50
    PyPlot.plt[:subplot](3, 2, 6 )
    PyPlot.plt[:axis]("on")
    h = PyPlot.plt[:hist](mc.qn,nbins, color = "g", alpha=0.75)
    PyPlot.plt[:xlabel]("Qn")     
    PyPlot.plt[:grid](true)
    
    nbins = 50
    PyPlot.plt[:subplot](3, 2, 5 )
    h = PyPlot.plt[:hist](mc.qc,nbins, color="g", alpha=0.75)
    PyPlot.plt[:grid](true)

    PyPlot.plt[:xlabel]("Qc") 
    PyPlot.plt[:savefig]("dbscan-abc.png")
    PyPlot.plt[:show]()
    
    ## Stats
    println("## Stats:")
    println("ϵ : ",mean(mc.eps)," +/- ", std(mc.eps))
    println("min_nei  : ",mean(mc.mne)," +/- ", std(mc.mne))
    println("min_clus  : ",mean(mc.mcl),"+/- ", std(mc.mcl))
    println("Qn  : ",mean(mc.qn)," +/- ", std(mc.qn)) 
    println("Qc  : ",mean(mc.qc)," +/- ", std(mc.qc)) 
    
    
    
end

### main 
minQ    = 2.4
minstars = 40
##
epsmean = 1.0
epsdisp = 1.5
min_nei   = 5
min_cl    = 10
ncoredisp = 5
##
nburnout  = 100
niter     = 100
pinit = abc(minQ, minstars, epsmean,epsdisp,min_nei, min_cl, ncoredisp, nburnout , niter)
##
main(pinit)